In [1]:
#import csv
#import re
#with open(r"D:\program_Lab_Report\machine_learning\100day\code\3_逻辑回归\SMSSpamCollection.csv",encoding="utf-8") as f:
#    with open(r"S_MSSpamCollection.csv",mode="w",encoding="utf-8",newline="")as f1:
#        pen=csv.writer(f1)
#        data=f.readlines()
#        for item in data:
#            item=item.split("\t")
#            item[1]=item[1].replace("\n","")
#            pen.writerow(item)

# 数据分析

In [2]:
import numpy as np 
import pandas as pd
#导入数据
df=pd.read_csv(r"D:\program_Lab_Report\machine_learning\100day\code\3_逻辑回归\S_MSSpamCollection.csv")
df.columns=["label","message"]
#查看标签列的基础信息
df.groupby('label').describe()  
#标签数值化
df['label_num']=df.label.map({'ham':0,'spam':1})
#信息数值化
df["message_len"]=df.message.map(len)
df.head()

,label,message,label_num,message_len
0,ham,Ok lar... Joking wif u oni...,0,29
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,155
2,ham,U dun say so early hor... U c already then say...,0,49
3,ham,"Nah I don't think he goes to usf, he lives aro...",0,61
4,spam,FreeMsg Hey there darling it's been 3 week's n...,1,147


# 数据预处理
* 1、移除所有标点符号
* 2、移除所有的通用词，如"the", "a"等。
* 在正常邮件和垃圾邮件中，标点符号和通用词汇的数量和类型是相似的，因此这些文本内容并不能起很好的区分作用，属于“无关特征”，需要利用预处理的手段将这些无关特征移除。

这里用到的是**NLTK**自然语言处理库

In [3]:
from sklearn.model_selection import train_test_split  #数据划分
import nltk
from nltk.corpus import stopwords
import string

nltk.download("stopwords")  
stopword=stopwords.words('english')

def text_process(message):
    string_message=[char for char in message if char not in string.punctuation]  #strin.punctuation是特殊符号
    string_message=''.join(string_message)
    string_message=' '.join([word for word in string_message.split() if word not in stopword+['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure']])
    return string_message
    
df["clean_message"]=df.message.map(text_process)
#del df["new_message"]
df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jie'jie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,label,message,label_num,message_len,clean_message
0,ham,Ok lar... Joking wif u oni...,0,29,Ok lar Joking wif oni
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,155,Free entry wkly comp win FA Cup final tkts 21s...
2,ham,U dun say so early hor... U c already then say...,0,49,U dun say early hor U c already say
3,ham,"Nah I don't think he goes to usf, he lives aro...",0,61,Nah I think goes usf lives around though
4,spam,FreeMsg Hey there darling it's been 3 week's n...,1,147,FreeMsg Hey darling 3 weeks word back Id like ...


# 将数据向量化
*  CountVectorizer统计词频矩阵

    CountVectorizer类会将文本中的词语转换为词频矩阵，例如矩阵中包含一个元素a[i][j]，它表示j词在i类文本下的词频。它通过fit_transform函数首先统计有多少词语出现，然后计算词语出现的次数，形成词频矩阵。训练集用fit_transform方法，测试集用transfrom方法。
* from sklearn.feature_extraction.text import TfidfTransformer
* x=TfidfTransformer().fit_transform(x)

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
x=df["clean_message"]
y=df.iloc[:,2].values
cv=CountVectorizer()
x=cv.fit_transform(x)
#print(cv.get_feature_names())   # 列表形式呈现文章生成的词典
#print(cv.vocabulary_)            #字典形式呈现，key：词，value:词频
'''将数据转化为稀疏矩阵'''
x=x.toarray() 
#划分数据
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=20)

# 模型训练

In [5]:
from sklearn.linear_model import LogisticRegression

log_model=LogisticRegression(solver='liblinear')
log_model.fit(x_train,y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

# 模型预测+评估

In [6]:
test_pred=log_model.predict(x_test)
print(test_pred)
test_score=log_model.score(x_test,y_test)
print("测试集准确率="+str(test_score))

train_pred=log_model.predict(x_train)
print(test_pred)
train_score=log_model.score(x_train,y_train)
print("测试集准确率="+str(test_score))


[0 0 0 ... 0 0 0]
测试集准确率=0.9856502242152466
[0 0 0 ... 0 0 0]
测试集准确率=0.9856502242152466
